<a href="https://colab.research.google.com/github/Salma-Jamal/Prepare-Dataset-For-Ner/blob/main/prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import ast
import re 
import string
import numpy as np

In [20]:
data  = pd.read_csv('/content/train.csv')

In [21]:
data.annotations = data.annotations.apply(lambda annotations : ast.literal_eval(annotations))

In [22]:
data.head()

,text,annotations
0,اريد تحويل 70 دقيقه لرقم 01100000011 من 011000...,"[{'start': 25, 'end': 36, 'entity': 'phonenumb..."
1,طيب لو عايز احول من باقة الانترنت بتاعتى لرقم ...,"[{'start': 46, 'end': 57, 'entity': 'phonenumb..."
2,ممكن أعرف متبقيلي ميجابايتس قد ايه؟,"[{'start': 18, 'end': 27, 'entity': 'type'}]"
3,شو سعر دقيقة التجوال فى السعودية,"[{'start': 24, 'end': 32, 'entity': 'country'}..."
4,عاوز اضيف دقايق تجوال للرقم ده 01100000011,"[{'start': 31, 'end': 42, 'entity': 'phonenumb..."


In [23]:
data.text.to_list()[0] , data.annotations.to_list()[0] 

('اريد تحويل 70 دقيقه لرقم 01100000011 من 01100000015',
 [{'end': 36, 'entity': 'phonenumber', 'start': 25},
  {'end': 51, 'entity': 'phonenumber', 'start': 40}])

In [24]:
def prepare_dataset(sentence,annotations,sen_num):
  lst = {}
  keyorder = sentence.split()
  for i in annotations:
    sen = sentence[i['start']:i['end']].split()
    if len(sen) > 1:
      for l in range(len(sen)):
        if l == 0:
          lst[sen[l]] = 'B-'+i['entity']
        elif l == len(sen)-1:
          lst[sen[l]] = 'E-'+i['entity']
        else:
          lst[sen[l]] ='I-'+i['entity']

    else:
      lst["".join(sen)] = i['entity']

  for word in sentence.split():
    if word not in lst.keys() and word+' ' not in lst.keys():
      lst[word] = 'O'     
  lst = sorted(lst.items(), key=lambda i:keyorder.index(i[0]))
  lst = [('sent'+str(sen_num),) + i for i in lst]
  return lst

In [25]:
%%time
for i in range(len(data.text.to_list())):
  lst = prepare_dataset(data.text.to_list()[i],data.annotations.to_list()[i],i+1)
  if i == 0:
    NER_Data = pd.DataFrame(lst, columns=["Sentence","Word", "Tag"])
  else:
    df2 = pd.DataFrame(lst, columns=["Sentence","Word", "Tag"])
    NER_Data= NER_Data.append(df2)

CPU times: user 327 ms, sys: 5.59 ms, total: 333 ms
Wall time: 349 ms


In [28]:
NER_Data.head()

,Sentence,Word,Tag
0,sent1,اريد,O
1,sent1,تحويل,O
2,sent1,70,O
3,sent1,دقيقه,O
4,sent1,لرقم,O


- I- prefix before a tag indicates that the tag is inside a chunk. 
- O tag indicates that a token belongs to no chunk. 
- B- prefix before a tag indicates that the tag is the beginning of a chunk 
- E- prefix before a tag indicates that the tag is the ending of a chunk

In [32]:
NER_Data[NER_Data.Sentence == 'sent1']

,Sentence,Word,Tag
0,sent1,اريد,O
1,sent1,تحويل,O
2,sent1,70,O
3,sent1,دقيقه,O
4,sent1,لرقم,O
5,sent1,01100000011,phonenumber
6,sent1,من,O
7,sent1,01100000015,phonenumber


In [33]:
NER_Data[NER_Data.Sentence == 'sent243']

,Sentence,Word,Tag
0,sent243,عايز,O
1,sent243,اسافر,O
2,sent243,المملكة,B-country
3,sent243,العربية,I-country
4,sent243,السعودية,E-country
